In [2]:
import tensorflow as tf
print(tf.VERSION)
#import tensorflow.contib.slim
import sys
import os
import cv2
from mxnet import gluon, image, nd
from mxnet.gluon import data as gdata, utils as gutils
import d2l
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from tensorflow.python.framework import ops
import mxnet
print("DONE ALL")

voc_dir = "C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012"

def read_voc_images(root=voc_dir, is_train=True):
    txt_fname = '%s/ImageSets/Segmentation/%s' % (
        root, 'train.txt' if is_train else 'val.txt')
    with open(txt_fname, 'r') as f:
        images = f.read().split()
    features, labels = [None] * len(images), [None] * len(images)
    for i, fname in enumerate(images):
        features[i] = image.imread('%s/JPEGImages/%s.jpg' % (root, fname))
        labels[i] = image.imread(
            '%s/SegmentationClass/%s.png' % (root, fname))

    return features, labels

#train_features, train_labels = read_voc_images()



1.9.0
DONE ALL


In [ ]:
import tensorflow as tf
import sys
import os
import cv2
from mxnet import gluon, image, nd
from mxnet.gluon import data as gdata, utils as gutils
import d2l
import numpy as np
import sklearn
import mxnet

voc_dir_features = "C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012\\"
voc_dir_labels = "C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012\\"

def read_voc_images(root=voc_dir, is_train=True):
    txt_fname = '%s/ImageSets/Segmentation/%s' % (
        root, 'train.txt' if is_train else 'val.txt')
    with open(txt_fname, 'r') as f:
        images = f.read().split()
    features, labels = [None] * len(images), [None] * len(images)
    total_features=0
    for i, fname in enumerate(images):
        labels[i] =  cv2.imread(os.path.join(voc_dir_labels,'%s/SegmentationClass/%s.jpg' % (root, fname)))
        features[i] = cv2.imread(os.path.join(voc_dir_features,'%s/JPEGImages/%s.jpg' % (root, fname))) 
        if ((np.ndim(features[i]))>2) and ((np.ndim(labels[i]))>2):
            features[i] = cv2.imread(os.path.join(voc_dir_features,'%s/JPEGImages/%s.jpg' % (root, fname))) 
            labels[i] =  cv2.imread(os.path.join(voc_dir_labels,'%s/SegmentationClass/%s.jpg' % (root, fname)))
            total_features = total_features + i
        else:  # in the interest in keeping the output clean...
            pass
    for i in range(20):
        print(i)
        print(np.ndim(features[i]))
        print(features[i].shape)
        print(np.ndim(labels[i]))
        print(labels[i].shape)
    print(total_features)
    print(len(features))
    print(len(labels))
    return features, labels

train_features, train_labels = read_voc_images()

In [ ]:
n = 20
imgs = train_features[1424+n:1444+n] + train_labels[1424+n:1444+n]
img = train_features[1000] + train_labels[1000]

d2l.show_images(imgs, 2, n);


In [ ]:
def resize(features,labels):
    feature=[]
    label=[]
    for i in range(1464):
        print(i)
        feature[i] = features[i].resize((width, height), Image.BICUBIC)  
        label[i] = labels[i].resize((width, height), Image.BICUBIC)  
        print(i)
    return features,labels

train_features, train_labels = resize(train_features, train_labels)

In [ ]:
def resize(features,labels):
    feature=[]
    label=[]
    for i in range(1464):
        print(i)
        feature.append(cv2.resize(cv2.UMat(np.array(features[i]), dtype = np.float32),(512,512)))
        #label.append(cv2.resize(cv2.UMat(np.array(labels[i]), dtype = np.float32),(512,512)))
        #feature.append(features[i])
        print(i)
    return features,labels

train_features, train_labels = resize(train_features, train_labels)
#n = 20
#imgs = train_features[1424+n:1444+n] + train_labels[1424+n:1444+n]
#img = train_features[1000] + train_labels[1000]

#d2l.show_images(imgs, 2, n);

In [3]:
txt_fname = 'C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012\\ImageSets\\Segmentation\\train.txt'

In [4]:
with open(txt_fname, 'r') as f:
        images = f.read().split()

In [5]:
len(images)

1464

In [6]:
label =  cv2.imread('C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012\\SegmentationClass\\' + images[0] + '.png')

In [7]:
np.shape(label)

(281, 500, 3)

In [ ]:
labels = []
features = []
for i in range(len(images)):
    print(i)
    labels.append(cv2.resize(cv2.imread('C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012\\SegmentationClass\\' + images[i] + '.png'), (512, 512)))
    features.append(cv2.resize(cv2.imread('C:\\Users\\bhatt\\Desktop\\Jupyter Notebook\\VOCdevkit\\VOC2012\\JPEGImages\\' + images[i] + '.jpg'), (512, 512)))

0
1
2
3
4
5


In [ ]:
len(features)

In [ ]:
plt.imshow(features[55])

In [ ]:
plt.imshow(labels[55])

In [ ]:
plt.imshow(features[316])
np.shape(labels[200])

In [ ]:
plt.imshow(labels[316])

In [ ]:
def create_placeholders(n_H0, n_W0, n_C0, n_H1,n_W1,n_C1):

    X = tf.placeholder(tf.float32, shape=[None, n_H0, n_W0, n_C0])
    Y = tf.placeholder(tf.float32, shape=[None, n_H1,n_W1,n_C1])    
    return X, Y

In [ ]:
def initialize_parameters():
 
    tf.set_random_seed(1)                              # so that your "random" numbers match ours  
    W1 = tf.get_variable("W1", [6, 6, 3, 60], initializer =  tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [3, 3, 60, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable("W3", [2, 2, 25, 3], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W4 = tf.get_variable("W4", [2, 2, 3, 3], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W5 = tf.get_variable("W5", [6, 6, 3, 3], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W6 = tf.get_variable("W6", [6, 6, 3, 3], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "W4": W4,
                  "W5": W5,
                  "W6": W6
                 }
    return parameters

In [ ]:
def forward_propagation(X, parameters):
     
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W6 = parameters['W6']
    W5 = parameters['W5']
    # CONV2D: 
    Z1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding = 'VALID')
    # RELU
    A1 = tf.nn.relu(Z1)
    # MAXPOOL:
    P1 = tf.nn.max_pool(A1, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    # CONV2D:
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,2,2,1], padding = 'VALID')
    # RELU
    A2 = tf.nn.relu(Z2)
    # MAXPOOL:
    P2 = tf.nn.max_pool(A2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    # FLATTEN
    Z3 = tf.nn.conv2d(P2,W3, strides = [1,2,2,1], padding = 'SAME')
    # RELU
    A3 = tf.nn.relu(Z3)
    print(np.shape(A3))
    Z4 = tf.contrib.slim.conv2d_transpose(A3,42, kernel_size = [3,3], stride=[2,2],activation_fn=tf.nn.relu)
    print("A4")
    A4 = tf.nn.relu(Z4)
    print(np.shape(A4))
    Z6 = tf.contrib.slim.conv2d_transpose(A4,8,[258,258],[2,2],activation_fn=tf.nn.relu)
    A6 = tf.nn.relu(Z6)
    print("A6")
    print(np.shape(A6))
    Z7 = tf.contrib.slim.conv2d_transpose(A6,8,[258,258],[2,2],activation_fn=tf.nn.relu)
    A7 = tf.nn.relu(Z7)
    print("A7")
    print(np.shape(A7))
    Z5 = tf.contrib.slim.conv2d_transpose(A7,3,[512,512],[2,2],activation_fn=tf.nn.relu)
    A5 = tf.nn.relu(Z5)
    print("A5")
    print(np.shape(A5))
    return A5

In [ ]:
def compute_cost(A5, Y):

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = A5, labels = Y))    
    return cost

In [ ]:
def model(X_train, Y_train, learning_rate = 0.009,num_epochs = 100, minibatch_size = 24, print_cost = True):
      
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3      
    m=1464                                    # to keep results consistent (numpy seed)
#     (n_H0, n_W0, n_C0) = np.shape(X_train)          
#     (n_H1,n_W1,n_C1) = np.shape(Y_train)   
    n_H0 = 512
    n_W0 = 512
    n_C0 = 3
    n_H1 = 512
    n_W1 = 512
    n_C1 = 3
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of the correct shape
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_H1,n_W1,n_C1)
    # Initialize parameters
    parameters = initialize_parameters()     
    # Forward propagation: Build the forward propagation in the tensorflow graph
    A5 = forward_propagation(X, parameters)    
    # Cost function: Add cost function to tensorflow graph
    cost = compute_cost(A5, Y)    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
    seed = 0
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            # minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            print(num_minibatches)
            print(epoch)
            for minibatch in range(num_minibatches):

                # Select a minibatch
                minibatch_X = X_train[(seed*minibatch_size):((seed*minibatch_size)+minibatch_size)]
                minibatch_Y = Y_train[(seed*minibatch_size):((seed*minibatch_size)+minibatch_size)]
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y})
                
                minibatch_cost += temp_cost / num_minibatches
            
            seed = seed + 1    

            # Print the cost every epoch
            #if print_cost == True and epoch % 5 == 0:
            print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            # if print_cost == True and epoch % 1 == 0:
            costs.append(minibatch_cost)
        
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(A5, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        # train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        # test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        # print("Train Accuracy:", train_accuracy)
        # print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy, parameters

In [ ]:
_,_,parameters = model(features,labels)